# ZILLOW

# Acquire

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from env import host, user, password
from sklearn.impute import KNNImputer

In [2]:
# Creating a string that connects me to MySQLWorkbench

def get_connection(db, user=user, host=host, password=password):
    '''
    get_connection uses login info from env.py file to access Codeup db.
    It takes in a string name of a database as an argument.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    

**I imported specific usefulcolumns based on the experience I had using this dataframe in previous assignment from the data science team**

In [3]:
# Getting the data from the Codeup database

def get_zillow_data():
    '''
    zillow_data() gets the zillow (only properties_2017 table) data from Codeup db, then writes it to a csv file,
    and returns the DF.
    '''
    # Creating a SQL query
    sql_query = '''
                SELECT
      properties_2017.parcelid,
      bathroomcnt,
      bedroomcnt,
      propertylandusedesc,
      calculatedfinishedsquarefeet,
      fips,
      latitude,
      longitude,  
      lotsizesquarefeet,
      yearbuilt, 
      structuretaxvaluedollarcnt,
      taxvaluedollarcnt, 
      landtaxvaluedollarcnt,
      taxamount,                      
      predictions_2017.logerror,                       
      predictions_2017.transactiondate
   FROM properties_2017
   JOIN predictions_2017 USING(parcelid)
   LEFT JOIN airconditioningtype USING(airconditioningtypeid)
   LEFT JOIN architecturalstyletype USING(architecturalstyletypeid)
   LEFT JOIN buildingclasstype USING(buildingclasstypeid)
   LEFT JOIN heatingorsystemtype USING(heatingorsystemtypeid)
   LEFT JOIN propertylandusetype USING(propertylandusetypeid)
   LEFT JOIN storytype USING(storytypeid)
   LEFT JOIN typeconstructiontype USING(typeconstructiontypeid)
   WHERE
      latitude IS NOT NULL
      AND longitude IS NOT NULL
      AND propertylandusedesc = 'Single Family Residential';
                '''
    
    # Reading in the DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('zillow'))
    return df

In [4]:
# Assigning the dataframe to a variable and displaying of it to have a first look

houses = get_zillow_data()
houses.head()

,parcelid,bathroomcnt,bedroomcnt,propertylandusedesc,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,logerror,transactiondate
0,14297519,3.5,4.0,Single Family Residential,3100.0,6059.0,33634931.0,-117869207.0,4506.0,1998.0,485713.0,1023282.0,537569.0,11013.72,0.025595,2017-01-01
1,17052889,1.0,2.0,Single Family Residential,1465.0,6111.0,34449266.0,-119281531.0,12647.0,1967.0,88000.0,464000.0,376000.0,5672.48,0.055619,2017-01-01
2,14186244,2.0,3.0,Single Family Residential,1243.0,6059.0,33886168.0,-117823170.0,8432.0,1962.0,85289.0,564778.0,479489.0,6488.30,0.005383,2017-01-01
3,12177905,3.0,4.0,Single Family Residential,2376.0,6037.0,34245180.0,-118240722.0,13038.0,1970.0,108918.0,145143.0,36225.0,1777.51,-0.103410,2017-01-01
4,12095076,3.0,4.0,Single Family Residential,2962.0,6037.0,34145202.0,-118179824.0,63000.0,1950.0,276684.0,773303.0,496619.0,9516.26,-0.001011,2017-01-01


In [5]:
# Displaying number of rows and columns

houses.shape

(52442, 16)

In [6]:
# Writing houses to csv on my computer

houses.to_csv('houses.csv')

In [7]:
# Displaying some general information about the data

houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52442 entries, 0 to 52441
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52442 non-null  int64  
 1   bathroomcnt                   52442 non-null  float64
 2   bedroomcnt                    52442 non-null  float64
 3   propertylandusedesc           52442 non-null  object 
 4   calculatedfinishedsquarefeet  52360 non-null  float64
 5   fips                          52442 non-null  float64
 6   latitude                      52442 non-null  float64
 7   longitude                     52442 non-null  float64
 8   lotsizesquarefeet             52073 non-null  float64
 9   yearbuilt                     52326 non-null  float64
 10  structuretaxvaluedollarcnt    52358 non-null  float64
 11  taxvaluedollarcnt             52441 non-null  float64
 12  landtaxvaluedollarcnt         52441 non-null  float64
 13  t

In [8]:
# Displaying statistical analysis of the data through.descibe()
# with astype('int64') to cut through all the noise of the float data
# displayed when using only 'houses.describe().T'

houses.describe().T.astype('int64')

,count,mean,std,min,25%,50%,75%,max
parcelid,52442,12999117,3411411,10711855,11510295,12577655,14129510,167687839
bathroomcnt,52442,2,1,0,2,2,3,18
bedroomcnt,52442,3,0,0,3,3,4,14
calculatedfinishedsquarefeet,52360,1922,1004,128,1268,1659,2306,21929
fips,52442,6049,21,6037,6037,6037,6059,6111
latitude,52442,34022538,273976,33340620,33826909,34023771,34187149,34818767
longitude,52442,-118194184,356661,-119475416,-118400683,-118153075,-117929152,-117554636
lotsizesquarefeet,52073,11339,86746,236,5583,6840,8798,6971010
yearbuilt,52326,1963,23,1878,1950,1961,1979,2016
structuretaxvaluedollarcnt,52358,196531,254338,129,77071,131807,226319,9164901


**<font color = 'blue'>Observations</font>**

><font color = 'blue'>There are some null values</font>

><font color = 'blue'>The counties are designated by their respective numbers and need to be renamed</font>

><font color = 'blue'>Many columns need to be renamed for better readability</font>

**<font color = 'blue'>Actions</font>**

><font color = 'blue'>Drop nulls</font>

><font color = 'blue'>Determine a threshhold to drop null value-ridden columns and drop them</font>

><font color = 'blue'>Detect and remove outliers if necessary unless going for MVP then wait to get MVP</font>

><font color = 'blue'>Create a county column with boolean and names for exploration and scaling</font>

><font color = 'blue'>Create a column for the houses' age</font>

><font color = 'blue'>Rename difficult to read columns</font>

# Prepare

In [10]:
# Dropping duplicates in rows if there are any and displaying the new count of columns and rows
houses.drop_duplicates(keep='first')

houses.shape

(52442, 16)

><font color = 'blue'>**There are no duplicates in the data**</font>

In [11]:
# Trying my function

def multi_frequency(df,vars):
    '''multi_frequency takes a dataframe in *arg and a *kwarg in the form of a list of columns
    and return a dataframe with the count and the frequency of the data
    '''
    frequency=df[vars].isnull().sum()
    percentage=df[vars].isnull().sum()*100/(len(df))
    df=pd.concat([frequency,percentage], axis=1, keys=['num_rows_missing', 'pct_rows_missing'])
    return df

In [12]:
multi_frequency(houses, houses.columns).sort_values(by='pct_rows_missing', ascending=False)

,num_rows_missing,pct_rows_missing
lotsizesquarefeet,369,0.703634
yearbuilt,116,0.221197
structuretaxvaluedollarcnt,84,0.160177
calculatedfinishedsquarefeet,82,0.156363
taxamount,4,0.007627
taxvaluedollarcnt,1,0.001907
landtaxvaluedollarcnt,1,0.001907
parcelid,0,0.000000
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000


**lotsizesquarefeet, yearbuilt, structuretaxvaluedollarcnt, and calculatedfinishedsquarefeet have the most nulls. I will use KNNImputer to handle them and drop the null rows in taxamount, taxvaluedollarcnt, and landtaxvaluedollarcnt.**

In [14]:
def handle_missing_values(df):
    imputer = KNNImputer(n_neighbors=1)
    imputer.fit_transform(houses[['lotsizesquarefeet',
                                  'yearbuilt',
                                  'structuretaxvaluedollarcnt',
                                  'calculatedfinishedsquarefeet']])
    df = df.dropna()
    return df

In [17]:
houses2 = handle_missing_values(houses)

In [18]:
multi_frequency(houses2, houses2.columns).sort_values(by='pct_rows_missing', ascending=False)

,num_rows_missing,pct_rows_missing
parcelid,0,0.0
bathroomcnt,0,0.0
bedroomcnt,0,0.0
propertylandusedesc,0,0.0
calculatedfinishedsquarefeet,0,0.0
fips,0,0.0
latitude,0,0.0
longitude,0,0.0
lotsizesquarefeet,0,0.0
yearbuilt,0,0.0


In [19]:
houses2.shape

(51897, 16)

In [20]:
def handle_missing_values2(df):
    imputer = KNNImputer(n_neighbors=2)
    imputer.fit_transform(houses[['lotsizesquarefeet',
                                  'yearbuilt',
                                  'structuretaxvaluedollarcnt',
                                  'calculatedfinishedsquarefeet']])
    df = df.dropna()
    return df

In [21]:
houses3 = handle_missing_values(houses)

In [22]:
multi_frequency(houses3, houses3.columns).sort_values(by='pct_rows_missing', ascending=False)

,num_rows_missing,pct_rows_missing
parcelid,0,0.0
bathroomcnt,0,0.0
bedroomcnt,0,0.0
propertylandusedesc,0,0.0
calculatedfinishedsquarefeet,0,0.0
fips,0,0.0
latitude,0,0.0
longitude,0,0.0
lotsizesquarefeet,0,0.0
yearbuilt,0,0.0


In [ ]:
def prepare_zillow(df):
    '''
    Takes in the df and changes county numbers
    to a boolean column,  indicating county names 
    for properties and another one the age
    '''
# create df with counties as booleans
    county_df = pd.get_dummies(df.fips)
    county_df.columns = ['Los_Angeles', 'Orange', 'Ventura']
    df = pd.concat([df, county_df], axis = 1)
# calculate age of home
    df['age'] = 2017 - df.yearbuilt
    df.drop(columns=['yearbuilt'], inplace=True)
    return df

In [ ]:
# Displaying the columns and their the count and percent of missing rows ordered by percent in a descending order

multi_frequency(houses, houses.columns).sort_values(by='pct_rows_missing', ascending=False)

In [ ]:
# # For loop to rename duplicate columns (particularly if they have the same name and not the same content)

# cols=pd.Series(df.columns)
# for dup in df.columns[df.columns.duplicated(keep=False)]: 
#     cols[df.columns.get_loc(dup)] = ([dup + '_' + str(d_idx) 
#                                      if d_idx != 0 
#                                      else dup 
#                                      for d_idx in range(df.columns.get_loc(dup).sum())]
#                                     )
# df.columns=cols

In [ ]:
# def multi_frequency_rows(df,vars):
#     '''multi_frequency takes a dataframe in *arg and a *kwarg in the form of a list of columns
#     and return a dataframe with the count and the frequency of the data
#     '''
#     frequency=df[vars].isnull().sum(axis=1)
#     percentage=df[vars].isnull().sum(axis=1)*100/(len(df))
#     df=pd.concat([frequency,percentage], axis=1, keys=['num_entries_missing', 'pct_entries_missing'])
#     return df
# multi_frequency(houses, ['basementsqft'])

In [ ]:
# Percent of columns that is missing +30% of its entries

percent_nullvalues_columns = (len(frequency[frequency.pct_rows_missing >= 33.589411]))/len(frequency)

In [ ]:
percent_nullvalues_columns

**<font color = 'blue'>Observations</font>**

><font color = 'blue'>Nulls have been removed</font>
Now what to do with next?



**<font color = 'blue'>Actions</font>**

><font color = 'blue'>Using the data dictionary, I will examine each column to determine it nature and poetential usefulness</font>

><font color = 'blue'>Determine imputability or drop the rest of the null values but keep the column</font>

><font color = 'blue'>Split the data into train, validate, and test</font>

><font color = 'blue'>Determine scalability and scale if necessary and not time consuming</font>

## Takeaways
- 54.71% of the data is missing between 33% and 100% of its values
- This makes me think that the best cut off would be the 33% threshold
- Many columns are id columns and should be removed. They have no impact on the analysis of the data and teh dataframe already has parcelid as unique identifier. Here is the list of the columns to drop:
     'typeconstructiontypeid',
     'storytypeid',
     'heatingorsystemtypeid',
     'buildingclasstypeid',
     'architecturalstyletypeid',
     'airconditioningtypeid',
     'propertylandusetypeid',
     'id',
     'buildingqualitytypeid',
     'id_1'

# Prepare

## 1. Remove any properties that are likely to be something other than single unit properties.

**Single unit properties were selected in the SQL query. The code for it is shown in the SQL query**

## 2. Create a function that will drop rows or columns based on the percent of values that are missing

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;A. Handling missing values

In [ ]:
def examine_and_prepare()

In [ ]:
def _maybe_dedup_names(self, names):
    # see gh-7160 and gh-9424: this helps to provide
    # immediate alleviation of the duplicate names
    # issue and appears to be satisfactory to users,
    # but ultimately, not needing to butcher the names
    # would be nice!
    if self.mangle_dupe_cols:
        names = list(names)  # so we can index
        counts = {}

        for i, col in enumerate(names):
            cur_count = counts.get(col, 0)

            if cur_count > 0:
                names[i] = '%s.%d' % (col, cur_count)

            counts[col] = cur_count + 1

    return names

In [ ]:
figure, ax1 = plt.subplots()
ax1.plot(df.iloc[:,0],df.iloc[:,1],linewidth=0.5,zorder=1, label = )
ax1.plot(df.iloc[:,0],df.iloc[:,2],linewidth=0.5,zorder=1, label = )